In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.appName('Practice')\
.master("local[*]")\
.getOrCreate()

24/08/19 14:25:48 WARN Utils: Your hostname, PC-390 resolves to a loopback address: 127.0.1.1; using 192.168.15.11 instead (on interface enp4s0)
24/08/19 14:25:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/19 14:25:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
import re
import pandas

In [5]:
# Read the  Data (Always give full path in Spark)
spark.read.text("Data/access_log_data")



df= spark.read.text("Data/access_log_data")

df.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


In [6]:
df.count()

1569898

In [7]:
# Partitioning the data
df.rdd.getNumPartitions()

12

In [8]:
sample = df.take(10)

In [9]:
sample

[Row(value='in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839'),
 Row(value='uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0'),
 Row(value='uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0'),
 Row(value='uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0'),
 Row(value='uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0'),
 Row(value='ix-esc-ca2-07.ix.netcom.com - - [01/Aug/1995:00:00:09 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1713'),
 Row(value='uplherc.upl.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/WORLD-logosmall.gif HTTP/1.0" 304 0'),
 Row(value='slppp6.intermind.net - - [01/Aug/1995:00:00:10 -0400] "GET /history/skylab/skylab.html HTTP/1.0" 200 1687'),
 Row(value='piweba4y.prodigy.com - - [01/Aug/1995:00:00:10 -0400] "GET /images/l

In [10]:
# Removing row function from data

sample = [item.value for item in sample]

In [11]:
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'

re.search(host_pattern, sample[0])

<re.Match object; span=(0, 18), match='in24.inetnebr.com '>

In [12]:
re.search(host_pattern, sample[0]).group()

'in24.inetnebr.com '

In [17]:
hosts= [re.search(host_pattern, item).group() for item in sample if re.search(host_pattern, item)]

In [18]:
hosts

['in24.inetnebr.com ',
 'uplherc.upl.com ',
 'uplherc.upl.com ',
 'uplherc.upl.com ',
 'uplherc.upl.com ',
 'ix-esc-ca2-07.ix.netcom.com ',
 'uplherc.upl.com ',
 'slppp6.intermind.net ',
 'piweba4y.prodigy.com ',
 'slppp6.intermind.net ']

In [20]:
# Extracting Time Stamps

ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'

# timestamps = [re.search(ts_pattern, item).group(1) for item in sample]
# timestamps

In [26]:
timestamp=[re.search(ts_pattern,item).group() for item in sample]

In [27]:
timestamp

['[01/Aug/1995:00:00:01 -0400]',
 '[01/Aug/1995:00:00:07 -0400]',
 '[01/Aug/1995:00:00:08 -0400]',
 '[01/Aug/1995:00:00:08 -0400]',
 '[01/Aug/1995:00:00:08 -0400]',
 '[01/Aug/1995:00:00:09 -0400]',
 '[01/Aug/1995:00:00:10 -0400]',
 '[01/Aug/1995:00:00:10 -0400]',
 '[01/Aug/1995:00:00:10 -0400]',
 '[01/Aug/1995:00:00:11 -0400]']

In [28]:
# extracting Protocol(get,post,put etc..)

method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
method_uri_protocol = [re.search(method_uri_protocol_pattern, item).groups()
                       if re.search(method_uri_protocol_pattern, item)
                       else "no uri"
                       for item in sample]
method_uri_protocol

[('GET', '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', 'HTTP/1.0'),
 ('GET', '/', 'HTTP/1.0'),
 ('GET', '/images/ksclogo-medium.gif', 'HTTP/1.0'),
 ('GET', '/images/MOSAIC-logosmall.gif', 'HTTP/1.0'),
 ('GET', '/images/USA-logosmall.gif', 'HTTP/1.0'),
 ('GET', '/images/launch-logo.gif', 'HTTP/1.0'),
 ('GET', '/images/WORLD-logosmall.gif', 'HTTP/1.0'),
 ('GET', '/history/skylab/skylab.html', 'HTTP/1.0'),
 ('GET', '/images/launchmedium.gif', 'HTTP/1.0'),
 ('GET', '/history/skylab/skylab-small.gif', 'HTTP/1.0')]

In [30]:
# Extracting Status Code
# 200 is success else every code is error i.e 404,403 etc
status_pattern = r'\s(\d{3})\s'
status = [re.search(status_pattern, item).group(1) for item in sample]
status

['200', '304', '304', '304', '304', '200', '304', '200', '200', '200']

In [31]:
# Extracting Content Size

contentsize_pattern = r'\s(\d+)$'
content_size = [re.search(contentsize_pattern, item).group(1) for item in sample]
content_size

['1839', '0', '0', '0', '0', '1713', '0', '1687', '11853', '9202']

In [36]:
from pyspark.sql.functions import regexp_extract, col

df.select(regexp_extract("value",host_pattern,1).alias("Host")).show()

+--------------------+
|                Host|
+--------------------+
|   in24.inetnebr.com|
|     uplherc.upl.com|
|     uplherc.upl.com|
|     uplherc.upl.com|
|     uplherc.upl.com|
|ix-esc-ca2-07.ix....|
|     uplherc.upl.com|
|slppp6.intermind.net|
|piweba4y.prodigy.com|
|slppp6.intermind.net|
|slppp6.intermind.net|
|ix-esc-ca2-07.ix....|
|slppp6.intermind.net|
|     uplherc.upl.com|
|        133.43.96.45|
|kgtyk4.kj.yamagat...|
|kgtyk4.kj.yamagat...|
|     d0ucr6.fnal.gov|
|ix-esc-ca2-07.ix....|
|     d0ucr6.fnal.gov|
+--------------------+
only showing top 20 rows



In [42]:
# Extracting All the Four Patterns


df.select(regexp_extract("value",host_pattern,1).alias("Host"),
         regexp_extract("value",ts_pattern,1).alias("Time Stamp"),
          # regexp_extract("value",url_pattern,1).alias("Status"),
         regexp_extract("value",status_pattern,1).alias("Status"),
          
         regexp_extract("value",contentsize_pattern,1).alias("Content Size"),
         ).show(10)

+--------------------+--------------------+------+------------+
|                Host|          Time Stamp|Status|Content Size|
+--------------------+--------------------+------+------------+
|   in24.inetnebr.com|01/Aug/1995:00:00...|   200|        1839|
|     uplherc.upl.com|01/Aug/1995:00:00...|   304|           0|
|     uplherc.upl.com|01/Aug/1995:00:00...|   304|           0|
|     uplherc.upl.com|01/Aug/1995:00:00...|   304|           0|
|     uplherc.upl.com|01/Aug/1995:00:00...|   304|           0|
|ix-esc-ca2-07.ix....|01/Aug/1995:00:00...|   200|        1713|
|     uplherc.upl.com|01/Aug/1995:00:00...|   304|           0|
|slppp6.intermind.net|01/Aug/1995:00:00...|   200|        1687|
|piweba4y.prodigy.com|01/Aug/1995:00:00...|   200|       11853|
|slppp6.intermind.net|01/Aug/1995:00:00...|   200|        9202|
+--------------------+--------------------+------+------------+
only showing top 10 rows



In [46]:
clean_df= df.select(regexp_extract("value",host_pattern,1).alias("Host"),
         regexp_extract("value",ts_pattern,1).alias("Time Stamp"),
          # regexp_extract("value",url_pattern,1).alias("Status"),
         regexp_extract("value",status_pattern,1).alias("Status"),
          
         regexp_extract("value",contentsize_pattern,1).alias("Content Size"),
         )

In [47]:
# Same code without Spark

clean_df.createTempView("Temp_Log_Data")

In [ ]:
spark.sql(" Select Status, COUNT(*) AS Number_of")